In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GroupShuffleSplit, GroupKFold, cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

plt.style.use("seaborn-v0_8")
sns.set(font_scale=1.2)
RANDOM_STATE = 42

print("Libraries imported successfully.")

Libraries imported successfully.


In [2]:
df = pd.read_csv("data/raw/Life Expectancy Data.csv")

df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True).str.replace(' ', '_')

target = "Life_expectancy"
group_col = "Country"
time_col = "Year"

df = df.dropna(subset=[target])

print(f"Data Loaded. Shape: {df.shape}")
df.head(3)

Data Loaded. Shape: (2928, 22)


,Country,Year,Status,Life_expectancy,Adult_Mortality,infant_deaths,Alcohol,percentage_expenditure,Hepatitis_B,Measles,...,Polio,Total_expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness_1-19_years,thinness_5-9_years,Income_composition_of_resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9


In [3]:
X = df.drop(columns=[target])
y = df[target]
groups = df[group_col]

# use GroupShuffleSplit to get 20% test
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train = groups.iloc[train_idx]

print(f"Train set shape: {X_train.shape}")
print(f"Test set shape:  {X_test.shape}")

# check leakage
overlapping = set(groups.iloc[train_idx]) & set(groups.iloc[test_idx])
print(f"Overlapping countries: {overlapping}")

Train set shape: (2336, 21)
Test set shape:  (592, 21)
Overlapping countries: set()
